# Linear Regression

[![Open In Colab](https://img.shields.io/badge/open%20in-Colab-b5e2fa?logo=googlecolab&style=flat-square&color=ffd670)](https://colab.research.google.com/github/tsonpham/ObsSeisHUS2025/blob/master/Day3/D3_Lab.ipynb)

Prepared by Thanh-Son Pham (thanhson.pham@anu.edu.au), April 2025.

<!-- [![Open In Colab](https://img.shields.io/badge/open%20in-Colab-b5e2fa?logo=googlecolab&style=flat-square&color=ffd670)]() -->

Acknowledgement: This notebook is modified from the original [tutorial](https://colab.research.google.com/github/inlab-geo/cofi-examples/blob/main/tutorials/linear_regression/linear_regression.ipynb) of the open-source tool [CoFI](https://cofi.readthedocs.io/en/latest/index.html), the Common Framework for Inference, to demonstrate several approaches to solve the parameter estimation problem to a polynomial.

We use the original functions from different software packages as a demonstration in this notebook. However, readers are encouraged to explore the original notebook to see how [CoFI](https://cofi.readthedocs.io/en/latest/index.html) bridges several concepts across inversioninto a unified framework.

---

## What we do in this notebook
Here we demonstrate several ways to approach a simple **linear regression** problem, where we fit a polynomial function to data using three different algorithms:

- by solution of a linear system of equations,
- by optimization of a data misfit function
- by Bayesian sampling of a Likelihood multiplied by a prior.

<!-- ---

## Learning outcomes

- A demonstration of running CoFI for a class of parameter fitting problem.  Example of a CoFI **template**.
- A demonstration of how CoFI may be used to  **experiment with different inference approaches** under a common interface.
- A demonstration of CoFI's **expandability** in that it may be used with pre-set, or user defined, misfits, likelihood or priors.   -->


In [ ]:
# Environment setup (uncomment code below)

# !pip install emcee corner

In [ ]:
#@title Run to activate retina display

%config InlineBackend.figure_format = "retina"
from matplotlib import rcParams

rcParams["savefig.dpi"] = 100
rcParams["figure.dpi"] = 100
rcParams["font.size"] = 10

---
## Preparation

Let's start with some `(x,y)` data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# here is some (x,y) data
data_x = np.array([1.1530612244897958, -0.07142857142857162, -1.7857142857142858,
                1.6428571428571423, -2.642857142857143, -1.0510204081632653,
                1.1530612244897958, -1.295918367346939, -0.806122448979592,
                -2.2755102040816326, -2.2755102040816326, -0.6836734693877551,
                0.7857142857142856, 1.2755102040816322, -0.6836734693877551,
                -3.2551020408163267, -0.9285714285714288, -3.377551020408163,
                -0.6836734693877551, 1.7653061224489797])

data_y = np.array([-7.550931153863841, -6.060810406314714, 3.080063056254076,
                -4.499764131508964, 2.9462042659962333, -0.4645899453212615,
                -7.43068837808917, 1.6273774547833582, -0.05922697815443567,
                3.8462283231266903, 3.425851020301113, -0.05359797104829345,
                -10.235538857712598, -5.929113775071286, -1.1871766078924957,
                -4.124258811692425, 0.6969191559961637, -4.454022624935177,
                -2.352842192972056, -4.25145590011172])
sigma = 1   # estimation on the data noise

And now lets plot the data.

In [ ]:
def plot_data(sigma=None):
    if(sigma is None):
        plt.scatter(data_x, data_y, color="lightcoral", label="observed data")
        plt.xlabel("X")
        plt.ylabel("Y")
    else:
        plt.errorbar(data_x, data_y, yerr=sigma, fmt='.', color="lightcoral", ecolor='lightgrey', ms=10, label='observed data with error')
plot_data()

---
## Problem description

To begin with, we will work with polynomial curves, $$y(x) = \sum_{j=0}^M m_j x^j\,.$$
Here, $M$ is the 'order' of the polynomial: if $M=1$ we have a straight line with 2 parameters, if $M=2$ it will be a quadratic with 3 parameters, and so on. The $m_j, (j=0,\dots M)$ are the 'model coefficients' that we seek to constrain from the data.

For this class of problem, the forward operator takes the following linear form:

$$\left(\begin{array}{c}y_0\\y_1\\\vdots\\y_N\end{array}\right) = \left(\begin{array}{ccc}1&x_0&x_0^2&x_0^3\\1&x_1&x_1^2&x_1^3\\\vdots&\vdots&\vdots\\1&x_N&x_N^2&x_N^3\end{array}\right)\left(\begin{array}{c}m_0\\m_1\\m_2\\m_3\end{array}\right)$$


This clearly has the required general form, $\mathbf{d} =G{\mathbf m}$.

where:

- $\textbf{d}$ is the vector of data values, ($y_0,y_1,\dots,y_N$);
- $\textbf{m}$ is the vector of model parameters, ($m_0,m_1,m_2, m_3$), where $M=3$;
- $G$ is the basis matrix (or design matrix) of this linear regression problem (also called the ***Jacobian*** matrix for this linear problem).

We have a set of noisy data values, $y_i (i=0,\dots,N)$, measured at known locations, $x_i (i=0,\dots,N)$, and wish to find the best fit degree 3 polynomial.

The function that generated our data is : $y=-6-5x+2x^2+x^3$, and we have added Gaussian random noise, ${N}(0,\sigma^2)$, with $\sigma=1.0$.

We now build the Jacobian/G matrix for this problem and define a forward function which simply multiplies $\mathbf m$ by $G$.

In [ ]:
nparams = 4 # Number of model parameters to be solved for

def jacobian(x=data_x, n=nparams):
    return np.array([x**i for i in range(n)]).T

def forward(model):
    return jacobian().dot(model)

def Cd_inv(sigma=sigma, ndata=len(data_x)):
    return 1/sigma**2 * np.identity(ndata)

Define the true model for later.

In [ ]:
# True model for plotting
x = np.linspace(-3.5,2.5)              # x values to plot
true_model = np.array([-6, -5, 2, 1])  # we know it for this case which will be useful later for comparison.

true_y = jacobian(x,4).dot(true_model) # y values for true curve

Now lets plot the data with the curve from the true polynomial coefficients.

In [ ]:
# Some plotting utilities
def plot_model(x,y, label, color=None):
    #x = np.linspace(-3.5,2.5)
    #y = jacobian(x).dot(model)
    plt.plot(x, y, color=color or "darkorange", label=label)
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.legend()

def plot_models(models, label="Posterior samples", color="seagreen", alpha=0.1):
    x = np.linspace(-3.5,2.5)
    G = jacobian(x)
    plt.plot(x, G.dot(models[0]), color=color, label=label, alpha=alpha)
    for m in models:
        plt.plot(x, G.dot(m), color=color, alpha=alpha)
    # plt.xlabel("X")
    # plt.ylabel("Y")
    plt.legend()

In [ ]:
plot_data(sigma=sigma)
plot_model(x,true_y, "true model")

Now we have the data and the forward model we can start to try and estimate the coefficients of the polynomial from the data.

---
## 1. Linear system solver

Our first choice is to define a linear parameter estimation problem (without any regularization) to be infered with the standard least square (analytical) solution.

$$
m = (G^T C_d^{-1} G)^{-1} G^T C_d^{-1} d.
$$

In [ ]:
# Jacobian matrix corresponding to known data_x
G = jacobian()
# Analytical least square solution
L = G.T @ Cd_inv() @ G # @ is a short form for np.matmul, i.e., matrix multiplication
R = G.T @ Cd_inv() @ data_y
m = np.linalg.inv(L) @ R
print (m)

The analytical solution can be verified with the `scipy.linalg.lstsq` function.

In [ ]:
from scipy.linalg import lstsq

m = lstsq(G, data_y)
print(m[0])

Lets plot the solution for visual inspection.

In [ ]:
plot_data()
plot_model(x,jacobian(x).dot(m[0]), "linear system solver", color="seagreen")
plot_model(x,true_y, "true model", color="darkorange")

---
## 2. Optimizer

The same overdetermined linear problem, $\textbf{d} = G\textbf{m}$, with Gaussian data noise can also be solved by minimising the squares of the
residual of the linear equations, e.g. $\textbf{r}^T \textbf{C}_d^{-1}\textbf{r}$ where
$\textbf{r}=\textbf{d}-G\textbf{m}$. The above matrix solver solution gives us the best data fitting model, but a direct optimisation approach could also be used, say when the number of unknowns is large and we do not wish, or are unable to provide the Jacobian function.

So we use the `scipy.optimize.minimize` function with [Nelder-Mead](https://en.wikipedia.org/wiki/Nelder%E2%80%93Mead_method) derivative-free optimization to demonstrate this option.

In [ ]:
def misfit(m):
    return np.sum((jacobian() @ m - data_y)**2) / sigma**2

from scipy.optimize import minimize
res = minimize(misfit, np.ones(nparams), method='Nelder-Mead')

print(res)

Lets plot the optimal solution for visual inspection.

In [ ]:
plot_data()
plot_model(x,jacobian(x).dot(res.x), "optimization solution", color="cornflowerblue")
plot_model(x,true_y, "true model", color="darkorange")

### Challenge: Change the polynomial degree

Try and replace the 3rd order polynomial with a 1st order polynomial (i.e. $M=1$) by adding the required commands below. What does the plot looks like?

Start from code below:

```
G = jacobian(x=data_x, n=2)
# Analytical least square solution
L = <CHANGE ME>
R = <CHANGE ME>
m = np.linalg.inv(L) @ R

print("Inferred curve with n = 2 ")
plot_data()
plot_model(x,jacobian(x,2)@m, "optimization solution", color="cornflowerblue")
plot_model(x,true_y, "true model", color="darkorange")
```

In [ ]:
# Copy the template above, Replace <CHANGE ME> with your answer



In [ ]:
#@title Solution

# Jacobian matrix corresponding to known data_x
G = jacobian(x=data_x, n=2)
# Analytical least square solution
L = G.T @ Cd_inv() @ G # @ is a short form for np.matmul, i.e., matrix multiplication
R = G.T @ Cd_inv() @ data_y
m = np.linalg.inv(L) @ R

print("Inferred curve with n = 2 ")
plot_data()
plot_model(x,jacobian(x,2)@m, "optimization solution", color="cornflowerblue")
plot_model(x,true_y, "true model", color="darkorange")

-------

---
## 3. Bayesian sampling

### Prior probability

Bayesian sampling requires a prior probability density function. A common problem with polynomial coefficients as model parameters is that it is not at all obvious what a prior should be. Here we choose a uniform prior with specified bounds
$$
\begin{align*}
p({\mathbf m}) &= \frac{1}{V},\quad  l_i \le m_i \le u_i, \quad (i=1,\dots,M)\\
         &= 0, \quad {\rm otherwise},
\end{align*}
$$
where $l_i$ and $u_i$ are lower and upper bounds on the $i$th model coefficient.

It is convient to work with log probability in Bayesian sampling and assume $V=1$ as it's a constant,
$$
\begin{align*}
\log p({\mathbf m}) &= 0,\quad  l_i \le m_i \le u_i, \quad (i=1,\dots,M)\\
         &= -\infty, \quad {\rm otherwise},
\end{align*}
$$

Here use the uniform distribution with ${\mathbf l}^T = (-10.,-10.,-10.,-10.)$, and ${\mathbf u}^T = (10.,10.,10.,10.)$.

In [ ]:
m_lower_bound = np.ones(nparams) * (-10.)             # lower bound for uniform prior
m_upper_bound = np.ones(nparams) * 10                 # upper bound for uniform prior

def log_prior(model):    # uniform distribution
    for i in range(len(m_lower_bound)):
        if model[i] < m_lower_bound[i] or model[i] > m_upper_bound[i]: return -np.inf
    return 0.0 # model lies within bounds -> return log(1)

### Likelihood

Since data errors follow a Gaussian in this example, we can define a likelihood function, $p({\mathbf d}_{obs}| {\mathbf m})$.

$$
p({\mathbf d}_{obs} | {\mathbf m}) = \dfrac{1}{\sqrt{(2\pi)^N\det(C_D)}} \exp \left\{- \frac{1}{2} ({\mathbf d}_{obs}-{\mathbf d}_{pred}({\mathbf m}))^T C_D^{-1} ({\mathbf d}_{obs}-{\mathbf d}_{pred}({\mathbf m})) \right\}
$$

where ${\mathbf d}_{obs}$ represents the observed y values and ${\mathbf d}_{pred}({\mathbf m})$ are those predicted by the polynomial model $({\mathbf m})$. The Likelihood is defined as the probability of observing the data actually observed, given a model. In practice we usually only need to evaluate the log of the Likelihood, $\log p({\mathbf d}_{obs} | {\mathbf m})$.

To do so, we require the inverse data covariance matrix describing the statistics of the noise in the data, $C_D^{-1}$. For this problem the data errors are independent with identical standard deviation in noise for each datum. Hence $C_D^{-1} = \frac{1}{\sigma^2}I$ and $\det(C_D) = \sigma^{2N}$ and the likelihood function becomes
$$
p({\mathbf d}_{obs} | {\mathbf m}) = (2\pi \sigma^2)^{-\dfrac{N}{2}} \exp \left\{- \frac{1}{2} \sum_{i=1}^N\left(\dfrac{d_{obs}^i - d_{pred}^i}{\sigma}\right)^2\right\}.
$$

For convenience, the log probability is often used to simplify the calculation and avoid floating over/underflow errors,
$$
\log p({\mathbf d}_{obs} | {\mathbf m}) = - \frac{1}{2}\left(\log (N2\pi\sigma^2) + \sum_{i=1}^N\left(\dfrac{d_{obs}^i - d_{pred}^i}{\sigma}\right)^2\right).
$$

When the data noise amplitude, $\sigma$ is assumed or estimated a priori, the likelihood function can be simplified even further,
$$
\log p({\mathbf d}_{obs} | {\mathbf m}) = - \frac{1}{2}\sum_{i=1}^N\left(\dfrac{d_{obs}^i - d_{pred}^i}{\sigma}\right)^2.
$$

Note that the data vector $\mathbf y$ is ${\mathbf d}_{obs}$ in this context.

In [ ]:
sigma = 1.0                                     # common noise standard deviation
def log_likelihood(model):
    pred_y = forward(model)
    residual = data_y - pred_y
    return -0.5 * np.sum(residual**2 / sigma**2)

### Bayesian sampling

In this aproach we sample a probability distribution rather than find a single best fit solution. Bayes' theorem tells us the the posterior distribution is proportional to the likelihood and the prior.

$$p(\mathbf{m}|\mathbf{d}_{obs}) = K p(\mathbf{d}_{obs}|\mathbf{m})p(\mathbf{m})$$

where $K = \dfrac{1}{p(\mathbf{d}_{obs})}$ is some constant. Under the assumptions specified $p(\mathbf{m}|\mathbf{d}_{obs})$ gives a probability density of models that are supported by the data. We seek to draw random samples from $p(\mathbf{m}|\mathbf{d}_{obs})$ over model space and then to make inferences from the resulting ensemble of model parameters.

When dropping the constant, the log posterior probability is written simply as,
$$
\log p(\mathbf{m}|\mathbf{d}_{obs}) = \log p(\mathbf{d}_{obs}|\mathbf{m}) + \log p(\mathbf{m}),
$$
or explicitly,
$$
\begin{align*}
\log p({\mathbf m} | {\mathbf d}_{obs}) &= - \frac{1}{2}\sum_{i=1}^N\left(\dfrac{d_{obs}^i - d_{pred}^i}{\sigma}\right)^2,\quad  l_i \le m_i \le u_i, \quad (i=1,\dots,M)\\
         &= -\infty, \quad {\rm otherwise},
\end{align*}
$$

A proposed sample $\mathbf m$ will never be accepted if drawn outside the parameter boundaries and might accepted with some randomness.

In this example we make use of *The Affine Invariant Markov chain Monte Carlo (MCMC) Ensemble sampler*
[Goodman and Weare 2010](https://msp.org/camcos/2010/5-1/p04.xhtml) to sample the
posterior distribution of the model. (See more details about [emcee](https://emcee.readthedocs.io/en/stable/)).

In [ ]:
def log_probbability(model):
    return log_prior(model) + log_likelihood(model)

### Starting points for random walkers

Now we define some hyperparameters (e.g. the number of walkers and steps), and initialise
the random starting positions of walkers. We start all walkers in a small ball about a chosen point
$(0, 0, 0, 0)$.

In [ ]:
nwalkers = 32
ndim = nparams
nsteps = 10000
walkers_start = np.zeros(nparams) + 1e-4 * np.random.randn(nwalkers, ndim)

### Run McMC chain to sample the posterior probability
The ensemble sampler with 32 walkers is run for 10,000 steps, meaning there are 320,000 samples of the model space will be drawn from by the sampler.

In [ ]:
import emcee
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probbability)
output = sampler.run_mcmc(walkers_start, 10000, progress=True)

### Analyse the Bayesian sampling output
We use `arviz` package to inspect the analyse the output of a Bayesian inference.

In [ ]:
import arviz as az
labels = ["$m_0$", "$m_1$", "$m_2$","$m_3$"]
idata = az.from_emcee(sampler, var_names=labels)
idata.get("posterior")

First, we use `plot_trace` to inspect the evolution of each model components as from the sampling chains. 32 chains are overlapped.

In [ ]:
# a standard `trace` plot
axes = az.plot_trace(idata, backend_kwargs={"constrained_layout":True})
# add legends
for i, axes_pair in enumerate(axes):
    ax1 = axes_pair[0]
    ax2 = axes_pair[1]
    ax1.axvline(true_model[i], linestyle='dotted', color='red')
    ax1.set_xlabel("parameter value")
    ax1.set_ylabel("density value")
    ax2.set_xlabel("number of iterations")
    ax2.set_ylabel("parameter value")

Next, lets compute the autocorrelation time for each components in the chain. Read more [here](https://emcee.readthedocs.io/en/stable/tutorials/autocorr/#computing-autocorrelation-times). For this example, the autocorrelation time for all component is about $n_A = 60$.

In [ ]:
tau = sampler.get_autocorr_time()
print(f"autocorrelation time: {tau}")

It is common practice to derive the final posterior distribution by discarding the first $5 \times n_A = 300$ and thinning the chain at every $n_A/2= 30^{th}$ samples.

In [ ]:
flat_samples = sampler.get_chain(discard=300, thin=30, flat=True)

We use a `corner` plot to summary the empirically estimated distribution of all model parameter. The title of the diagonal panels show the median and 95\% confidence intervals.

In [ ]:
import corner
fig = corner.corner(flat_samples, labels=labels, truths=true_model,
                     quantiles=[0.025, 0.5, 0.975], show_titles=True,
                     title_kwargs={"fontsize": 12},
                     levels=(0.68, 0.95), alpha=0.1)
fig.set_size_inches(7, 7)
plt.show()

Now we plot the predicted curves for the posterior ensemble of solutions.

In [ ]:
inds = np.random.randint(len(flat_samples), size=100) # get a random selection from posterior ensemble
plot_data(sigma=sigma)
plot_models(flat_samples[inds])
plot_model(x,true_y, "True model", color="darkorange")

### Challenge 1: Change the prior model bounds

Replace the previous prior bounds to new values

The original  uniform bounds had  

${\mathbf l}^T = (-10.,-10.,-10.,-10.)$, and ${\mathbf u}^T = (10.,10.,10.,10.)$.

Lets replace with

${\mathbf l}^T = (-1.,-10.,-10.,-10.)$, and ${\mathbf u}^T = (2.,10.,10.,10.)$.

We have only changed the bounds of the first parameter. However since the true value of constant term was 6, these bounds are now inconsistent with the true model.

What does this do to the posterior distribution?

Start from the code template below:

```
m_lower_bound = <CHANGE ME>             # lower bound for uniform prior
m_upper_bound = <CHANGE ME>                 # upper bound for uniform prior

def log_prior(model):    # uniform distribution
    for i in range(len(m_lower_bound)):
        if model[i] < m_lower_bound[i] or model[i] > m_upper_bound[i]: return -np.inf
    return 0.0 # model lies within bounds -> return log(1)

######## re-run the sampler
sampler.reset()
sampler.run_mcmc(walkers_start, 10000, progress=True)

flat_samples = sampler.get_chain(discard=300, thin=30, flat=True)
inds = np.random.randint(len(flat_samples), size=100) # get a random selection from posterior ensemble

######## plot the results
print("Resulting samples with prior model lower bounds of <CHANGE ME>, upper bounds of <CHANGE ME>")
plot_data()
plot_models(flat_samples[inds])
plot_model(x, true_y, "True model", color="darkorange")
```

In [ ]:
# Copy the template above, Replace <CHANGE ME> with your answer



In [ ]:
#@title Click for solution

######## update the problem with the new prior bounds
m_lower_bound = np.array([-1,-10,-10,-10])             # lower bound for uniform prior
m_upper_bound = np.array([2,10,10,10])                 # upper bound for uniform prior

def log_prior(model):    # uniform distribution
    for i in range(len(m_lower_bound)):
        if model[i] < m_lower_bound[i] or model[i] > m_upper_bound[i]: return -np.inf
    return 0.0 # model lies within bounds -> return log(1)

######## re-run the sampler
sampler.reset()
sampler.run_mcmc(walkers_start, 10000, progress=True)

flat_samples = sampler.get_chain(discard=300, thin=30, flat=True)
inds = np.random.randint(len(flat_samples), size=100) # get a random selection from posterior ensemble

######## plot the results
print("Resulting samples with prior model lower bounds of [-1,-10,-10,-10], upper bounds of [2,10,10,10]")
plot_data(sigma=1)
plot_models(flat_samples[inds])
plot_model(x, true_y, "True model", color="darkorange")

Why do you think the posterior distribution looks like this?

### Challenge 2: Change the data uncertainty

To change the data uncertainty we increase `sigma` and then redefine the log-Likelihood.

Here we increase the assumed data standard deviation by a factor of of 50! So we are telling the inversion that the data are far less accurate than they actually are.

In [ ]:
sigma = 10.0 # common noise standard deviation

Lets return the prior to the original bounds.

In [ ]:
m_lower_bound = np.ones(nparams) * (-10.)             # lower bound for uniform prior
m_upper_bound = np.ones(nparams) * 10                 # upper bound for uniform prior

def log_prior(model):    # uniform distribution
    for i in range(len(m_lower_bound)):
        if model[i] < m_lower_bound[i] or model[i] > m_upper_bound[i]: return -np.inf
    return 0.0 # model lies within bounds -> return log(1)

Your challenge is then to tell CoFI that the Likelihood and prior have changed and then to rerun the sample, and plot results.

Feel free to start from the code below:

```
######## the likelihood function with the new Cdinv
def log_likelihood(model):
    pred_y = forward(model)
    residual = data_y - pred_y
    return <CHANGE ME>

######## re-run the sampler
sampler.reset()
sampler.run_mcmc(walkers_start, 10000, progress=True)

flat_samples = sampler.get_chain(discard=300, thin=30, flat=True)
inds = np.random.randint(len(flat_samples), size=100) # get a random selection from posterior ensemble

print("Resulting samples from changed data uncertainty")
plot_data()
plot_models(flat_samples[inds])
plot_model(x,true_y, "True model", color="darkorange")
```

In [ ]:
# Copy the template above, Replace <CHANGE ME> with your answer



The answer is in the next cells if you want to run them.

In [ ]:
#@title Click for solution

######## the likelihood function with the new Cdinv
def log_likelihood(model):
    pred_y = forward(model)
    residual = data_y - pred_y
    return -0.5 * np.sum(residual**2 / sigma**2)

######## re-run the sampler
sampler.reset()
sampler.run_mcmc(walkers_start, 10000, progress=True)

flat_samples = sampler.get_chain(discard=300, thin=30, flat=True)
inds = np.random.randint(len(flat_samples), size=100) # get a random selection from posterior ensemble

print("Resulting samples from changed data uncertainty")
plot_data(sigma=1)
plot_models(flat_samples[inds])
plot_model(x,true_y, "True model", color="darkorange")

### Challenge 3: Change the number of walkers / steps in the McMC algorithm (optional)

Now lets decrease the number of steps performed by the McMC algorithm. It will be faster but perform less exploration of the model parameters.

We suggest you reduce the number of steps taken by all 32 random walkers and see how it affects the posterior ensemble.

You can start from code template below:

```
nsteps = <CHANGE ME>               # instead of 10000
nwalkers = <CHANGE ME>             # instead of 32
# update start with the new number of walkers
walkers_start = np.zeros(nparams) + 1e-4 * np.random.randn(nwalkers, ndim)

# let's return to the old uncertainty settings
sigma = 1.0                                 # common noise standard deviation
Cdinv = np.eye(len(data_y))/(sigma**2)      # inverse data covariance matrix

# re-defining the sampler
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probbability)

######## re-run the sampler
sampler.run_mcmc(walkers_start, nsteps, progress=True)

######## CoFI InversionResult - plot result
flat_samples = sampler.get_chain(discard=300, thin=30, flat=True)
inds = np.random.randint(len(flat_samples), size=10) # get a random selection from posterior ensemble

print(f"Inference results from {nsteps} steps and {nwalkers} walkers")
plot_data()
plot_models(flat_samples[inds])
plot_model(x,true_y, "True model", color="darkorange")
```

In [ ]:
# Copy the template above, Replace <CHANGE ME> with your answer



In [ ]:
#@title Click for solution
# change number of steps
nsteps = 400              # instead of 10000

# change number of walkers
nwalkers = 30             # instead of 32
walkers_start = np.zeros(nparams) + 1e-4 * np.random.randn(nwalkers, ndim)

# let's return to the old uncertainty settings
sigma = 1.0                                 # common noise standard deviation

# re-defining the sampler
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probbability)

######## re-run the sampler
sampler.run_mcmc(walkers_start, nsteps, progress=True)

######## CoFI InversionResult - plot result
flat_samples = sampler.get_chain(discard=300, thin=30, flat=True)
inds = np.random.randint(len(flat_samples), size=10) # get a random selection from posterior ensemble

print(f"Inference results from {nsteps} steps and {nwalkers} walkers")
plot_data(sigma=1)
plot_models(flat_samples[inds])
plot_model(x,true_y, "True model", color="darkorange")

---
## Bonus project

Hello, thank you for being interested in the bonus project. Although everyone is encouraged to complete bonus projects, undergrad students will be considered for bonus points to their final exams. Best of luck!

*In this project, you are asked to write the correct form of the likelihood function to infer the noise amplitude hyperparameter, $\sigma$, together with the polinomial parameters.*

If you submit a working code towards completing the task, you will get 50% points. If the code produces correct outcome, you will get 75%. The minimal two-paragraphs on the motivation and additional thoughts will get you to 100%.

Please submit this jupyter notebook to the following form: https://forms.gle/L5QLLYMEnm277bTMA.

To start please look at the cell below for some hints.

In [ ]:
## polynomial parameters' bounds
m_lower_bound = np.ones(nparams) * (-10.)             # lower bound for uniform prior
m_upper_bound = np.ones(nparams) * 10                 # upper bound for uniform prior
## noise amplitude bounds
sigma_lower_bound = 0.0
sigma_upper_bound = 5.0

def log_prior_b(model):    # uniform distribution
    m = model[:-1] # model parameters
    sigma = model[-1] # noise amplitude
    for i in range(len(m_lower_bound)):
        if model[i] < m_lower_bound[i] or model[i] > m_upper_bound[i]: return -np.inf
    if sigma < sigma_lower_bound or sigma > sigma_upper_bound: return -np.inf
    return 0.0 # model lies within bounds -> return log(1)

def log_likelihood_b(model):
    # unpack the composite model including 4 polynomial parameters and 1 noise amplitude
    m = model[:-1] # model parameters
    sigma = model[-1] # noise amplitude
    ## ENTER YOUR CODE HERE

    # hints about returned expression
    return exp_term + norm_term

def log_probbability_b(model):
    return log_prior_b(model) + log_likelihood_b(model)

The following cell prepares the sampler with the correct configuration. No edits may be needed.

In [ ]:
#@markdown Set up and run the sampler
# change number of walkers
nwalkers = 32               # instead of 32
nsteps = 10000              # instead of 10000
# number of dimensions
ndim_b = nparams + 1

# starting walker position drwawn uniformly within the bounds
walkers_start_b = np.random.uniform(0, 1, size=(nwalkers, ndim_b))
walkers_start_b[:, :-1] = m_lower_bound + (m_upper_bound - m_lower_bound) * walkers_start_b[:, :-1]
walkers_start_b[:, -1] = sigma_lower_bound + (sigma_upper_bound - sigma_lower_bound) * walkers_start_b[:, -1]

######## re-run the sampler
sampler_b = emcee.EnsembleSampler(nwalkers, ndim_b, log_probbability_b)
sampler_b.run_mcmc(walkers_start_b, nsteps, progress=True)

######## autocorrelation time
tau_b = sampler_b.get_autocorr_time()
print(f"autocorrelation time: {tau_b}")

In [ ]:
#@markdown Plot marginalized sampled posterior
######## corner plot to show pair-marginalized posterior distributions
flat_samples = sampler_b.get_chain(discard=int(tau.max()), thin=int(tau.max()*.5), flat=True)
fig = corner.corner(flat_samples, labels=labels + ["$\sigma$"], truths=list(true_model) + [sigma],
                        quantiles=[0.025, 0.5, 0.975], show_titles=True,
                        levels=(0.68, 0.95), alpha=0.1)
fig.set_size_inches(7, 7)
plt.show()

In [ ]:
#@markdown Posterior polynomial fit
print(f"Inference results from {nsteps} steps and {nwalkers} walkers")
plot_data(sigma=1)
inds = np.random.randint(len(flat_samples), size=100)
plot_models(flat_samples[inds, :4])
plot_model(x,true_y, "True model", color="darkorange")